In [2]:
from datasets import load_dataset
from qdrant_client.http import models as rest

In [3]:
from qdrant_client import QdrantClient
from dotenv import load_dotenv
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os

In [4]:
dataset = load_dataset("pubmed_qa", "pqa_labeled")

In [5]:
import cohere

COHERE_API_KEY = "KqApPE9kQ9TSKjc8OkD0X9W2YEJlCkQBmg1Xb1tv"
cohere_client = cohere.Client(COHERE_API_KEY)

In [6]:
# Generating the embeddings with Cohere client library
embeddings = cohere_client.embed(
    texts=["A test sentence"],
    model="large",
)
vector_size = len(embeddings.embeddings[0])
print(vector_size) # output: 4096

unknown field: parameter compress is not a valid field


4096


In [7]:
len(embeddings.embeddings[0])

4096

In [8]:
answer_response = cohere_client.embed(
    texts=dataset["train"]["long_answer"],
    model="large",
)
vectors = [
    # Conversion to float is required for Qdrant
    list(map(float, vector)) 
    for vector in answer_response.embeddings
]
ids = [entry["pubid"] for entry in dataset["train"]]

unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field


unknown field: parameter compress is not a valid field


In [9]:
load_dotenv()

qdrant_url = os.getenv('QDRANT_URL')

client = QdrantClient(url=qdrant_url)


In [18]:
client.create_collection(
    collection_name="pubmed_qa_cohere_4096_v4",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

True

In [19]:
client.upsert(
    collection_name="pubmed_qa_cohere_4096_v4", 
    points=rest.Batch(
        ids=ids[0:100],
        vectors=vectors[0:100],
        payloads=list(dataset["train"])[:100],
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
question_response = cohere_client.embed(
    texts=dataset["train"]["question"][:100],
    model="large",
)

unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field


In [21]:
import tqdm

# Finding the position at which Qdrant provided the expected answer for each question. 
# That allows to calculate accuracy@k for different values of k.
k_max = 10
answer_positions = []
for embedding, pubid in tqdm.tqdm(zip(question_response.embeddings, ids)):
    response = client.search(
        collection_name="pubmed_qa_cohere_4096_v4",
        query_vector=embedding,
        limit=k_max,
    )

    answer_ids = [record.id for record in response]
    if pubid in answer_ids:
        answer_positions.append(answer_ids.index(pubid))
    else:
        answer_positions.append(-1)

100it [00:00, 232.93it/s]


In [25]:
# Prepared answer positions are being used to calculate different values of accuracy@k
for k in range(1, k_max + 1):
    correct_answers = len(
        list(
            filter(lambda x: 0 <= x < k, answer_positions)
        )
    )
    print(f"accuracy@{k} =", correct_answers / 100)

accuracy@1 = 0.94
accuracy@2 = 0.95
accuracy@3 = 0.95
accuracy@4 = 0.97
accuracy@5 = 0.98
accuracy@6 = 0.98
accuracy@7 = 0.98
accuracy@8 = 0.99
accuracy@9 = 0.99
accuracy@10 = 0.99
